In [22]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import os
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image


In [23]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [24]:
class CrackDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.labels_df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform
        self.label_map = {'Non-cracked': 0, 'Cracked': 1}

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = self.labels_df.iloc[idx, 0]
        label = self.label_map[self.labels_df.iloc[idx, 1]]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

In [15]:
# Load datasets
train_dataset = CrackDataset('../artifact_folder/train/labels.csv', '../artifact_folder/train/images', transform)
val_dataset = CrackDataset('../artifact_folder/val/labels.csv', '../artifact_folder/val/images', transform)
test_dataset = CrackDataset('../artifact_folder/test/labels.csv', '../artifact_folder/test/images', transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)


In [16]:
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 classes: Cracked / Non-cracked


/Users/nanphattongsirisukool/Documents/GitHub/Structural-Defects-Network-MLOps/env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/nanphattongsirisukool/Documents/GitHub/Structural-Defects-Network-MLOps/env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/nanphattongsirisukool/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:24<00:00, 1.89MB/s]


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [18]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()

        train_acc = correct / len(train_loader.dataset)
        print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Accuracy: {train_acc:.4f}")


In [20]:
def evaluate(model, val_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            correct += (outputs.argmax(1) == labels).sum().item()
    val_accuracy = correct / len(val_loader.dataset)
    print(f'Validation Accuracy: {val_accuracy:.4f}')


In [25]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)
evaluate(model, val_loader)


Epoch 1, Loss: 10595.1421, Accuracy: 0.6364
Epoch 2, Loss: 8369.4019, Accuracy: 0.7607
Epoch 3, Loss: 7632.7484, Accuracy: 0.7905
Epoch 4, Loss: 6988.4895, Accuracy: 0.8122
Epoch 5, Loss: 6575.0189, Accuracy: 0.8294
Epoch 6, Loss: 6196.2417, Accuracy: 0.8418
Epoch 7, Loss: 5777.6937, Accuracy: 0.8535
Epoch 8, Loss: 5029.1505, Accuracy: 0.8744
Epoch 9, Loss: 4336.9726, Accuracy: 0.8923
Epoch 10, Loss: 3418.7692, Accuracy: 0.9177
Validation Accuracy: 0.8733


In [27]:
torch.save(model.state_dict(), '../model/resnet_crack_classifier.pth')

In [28]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix
)

# Make sure model is in eval mode
model.eval()

# Consistent transform (same used during training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Evaluation function
def evaluate_model_on_split(split_name):
    base_path = f"../artifact_folder/{split_name}"
    labels_df = pd.read_csv(os.path.join(base_path, "labels.csv"))
    images_dir = os.path.join(base_path, "images")

    y_true, y_pred, y_score = [], [], []

    for _, row in labels_df.iterrows():
        img_path = os.path.join(images_dir, row["filename"])
        image = Image.open(img_path).convert("RGB")
        input_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            pred = output.argmax(dim=1).item()
            prob = torch.softmax(output, dim=1)[0][1].item()

        label = 1 if row["label"].lower() == "cracked" else 0
        y_true.append(label)
        y_pred.append(pred)
        y_score.append(prob)

    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1-Score": f1_score(y_true, y_pred, zero_division=0),
        "AUC-ROC": roc_auc_score(y_true, y_score),
        "Confusion Matrix": confusion_matrix(y_true, y_pred).tolist()
    }

# Run on all sets
for split in ['train', 'val', 'test']:
    print(f"📊 Evaluation for {split.upper()}")
    metrics = evaluate_model_on_split(split)
    for k, v in metrics.items():
        print(f"{k}: {v}")
    print()


📊 Evaluation for TRAIN
Accuracy: 0.934052333804809
Precision: 0.9719338715878508
Recall: 0.8939179632248939
F1-Score: 0.9312948977712285
AUC-ROC: 0.9850882188643193
Confusion Matrix: [[8265, 219], [900, 7584]]

📊 Evaluation for VAL
Accuracy: 0.8732913348389397
Precision: 0.5607424071991001
Recall: 0.7776911076443058
F1-Score: 0.6516339869281046
AUC-ROC: 0.9078117100283506
Confusion Matrix: [[6350, 781], [285, 997]]

📊 Evaluation for TEST
Accuracy: 0.8680926916221033
Precision: 0.5438100621820238
Recall: 0.7604743083003953
F1-Score: 0.6341463414634146
AUC-ROC: 0.8975873296663811
Confusion Matrix: [[6343, 807], [303, 962]]

